In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np


1. Data Loading
- 1.1 Exploration: Get to know the eight files using spreadsheet or text-editing software. Each file
was downloaded directly from the BLS and is otherwise unmodified. Use this to understand where
information is in the data, think about how they will all fit together, and identify issues that will
need to be resolved to load them

- Data Information (What are these datasets for, what do they tell us):
    - Civilian Employment: Tracks government civilian jobs by region and year.
    - Military Employment: Tracks armed forces employment, similarly split.
    - Metro vs. Micro: “Metro” covers major metropolitan statistical areas (MSAs), while “Micro” covers smaller micropolitan areas.
    - Time Split: Data is split into two time periods:
        - 1990–2000 (historical pre-standardization)
        - 2001–present (post-standardization with consistent formatting)


- How do they all fit together?: 
    - These files collectively cover civilian and military government employment across all U.S. metropolitan and micropolitan areas from 1990 to the present.

- Problems Identification (What are some problems within these dataset):
    - Inconsistent Headers/Formats within different files: Older files (1990–2000) may use different column names or structures than newer files.
    - Different FIPS Codes or Area Names: Naming mismatches or missing standardized geographic identifiers.

- 1.2 Loading, Joining, Reshaping: Load in all eight files and combine them into one “tidy” dataset,
where each row is uniquely identified by year, MSA Fips, and MSA Name, and there is a column
for each of military employment, civilian government employment, and total employment.
    - Audit your merging to be certain you are doing what you expect!
    - Before you join them into one final file and before you do any reshaping, your civilian government employment data should have 927 unique MSAs and 29 years, while your military and
total data should have 1,852 MSAs and the same 29 years. Why are there a different number
of MSAs in the two groups?
        - Ans: This is because Civilian government data only covers “Metropolitan Statistical Areas” (MSAs),
        while military data covers both Metropolitan and Micropolitan Statistical Areas.
        
    - Your final merge after reshaping to long will have 29 place-year observations in the civilian
government data that don’t match anything in the military and total data. What place is it?
        - Ans: the "District of Columbia Metropolitan Statistical Area. The District of Columbia (DC) is reported as an MSA in the civilian government employment data. However, in the military and total employment datasets, DC is not listed as its own MSA; its data is handled differently or may be aggregated under another region.

    - When you remove the unmatched observation and before you calculate any other columns,
your data should have 26,854 rows and 6 columns

In [24]:
# Military employment
df_mil_1990_metro   = pd.read_csv('military employment 1990-2000 Metro.csv',   skiprows=4)
df_mil_1990_micro   = pd.read_csv('military employment 1990-2000 Micro.csv',   skiprows=4)
df_mil_2001_metro   = pd.read_csv('military employment 2001-current Metro.csv', skiprows=4)
df_mil_2001_micro   = pd.read_csv('military employment 2001-current Micro.csv', skiprows=4)

# Civilian government employment
df_civ_1990_metro   = pd.read_csv('civilian government employment 1990-2000 Metro.csv',   skiprows=3)
df_civ_1990_micro   = pd.read_csv('civilian government employment 1990-2000 Micro.csv',   skiprows=3)
df_civ_2001_metro   = pd.read_csv('civilian government employment 2001-current Metro.csv', skiprows=3)
df_civ_2001_micro   = pd.read_csv('civilian government employment 2001-current Micro.csv', skiprows=3)


df_civ_2001_micro .head()

,GeoFips,GeoName,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,10100,"Aberdeen, SD (Micropolitan Statistical Area)",560.0,598.0,602.0,597.0,606.0,588.0,569.0,565.0,553.0,566.0,559.0,540.0,522.0,493.0,488.0,512.0,491.0,476.0
1,10140,"Aberdeen, WA (Micropolitan Statistical Area)",212.0,216.0,208.0,200.0,233.0,240.0,237.0,232.0,244.0,263.0,219.0,212.0,192.0,182.0,178.0,183.0,178.0,176.0
2,10220,"Ada, OK (Micropolitan Statistical Area)",242.0,242.0,234.0,230.0,223.0,222.0,188.0,181.0,185.0,199.0,175.0,170.0,160.0,158.0,162.0,164.0,158.0,150.0
3,10300,"Adrian, MI (Micropolitan Statistical Area)",225.0,230.0,232.0,224.0,222.0,227.0,226.0,221.0,226.0,237.0,206.0,199.0,189.0,185.0,190.0,195.0,195.0,193.0
4,10460,"Alamogordo, NM (Micropolitan Statistical Area)",1970.0,2022.0,1922.0,1889.0,1988.0,2007.0,1925.0,1892.0,1912.0,2006.0,2034.0,1917.0,1842.0,1711.0,1744.0,1758.0,1720.0,1690.0


In [25]:
def melt_military(df):
    """
    Reshape military employment data from wide to long format
    """
    # Get the year columns (all columns except GeoFips and GeoName)
    id_cols = ['GeoFips', 'GeoName']
    year_cols = [col for col in df.columns if col not in id_cols]
    
    # Melt the dataframe
    melted = pd.melt(df, 
                     id_vars=id_cols,
                     value_vars=year_cols,
                     var_name='Year',
                     value_name='military_emp')
    
    # Convert Year to integer and clean up
    melted['Year'] = pd.to_numeric(melted['Year'], errors='coerce')
    melted['military_emp'] = pd.to_numeric(melted['military_emp'], errors='coerce')
    
    return melted

def melt_civilian(df):
    """
    Reshape civilian employment data from wide to long format
    """
    # Get the year columns (all columns except GeoFips and GeoName)
    id_cols = ['GeoFips', 'GeoName']
    year_cols = [col for col in df.columns if col not in id_cols]
    
    # Melt the dataframe
    melted = pd.melt(df, 
                     id_vars=id_cols,
                     value_vars=year_cols,
                     var_name='Year',
                     value_name='civilian_emp')
    
    # Convert Year to integer and clean up
    melted['Year'] = pd.to_numeric(melted['Year'], errors='coerce')
    melted['civilian_emp'] = pd.to_numeric(melted['civilian_emp'], errors='coerce')
    
    return melted

In [26]:
# 1. Melt all
mil_long = pd.concat([melt_military(df) for df in [
    df_mil_1990_metro, df_mil_1990_micro, df_mil_2001_metro, df_mil_2001_micro]], ignore_index=True)
civ_long = pd.concat([melt_civilian(df) for df in [
    df_civ_1990_metro, df_civ_1990_micro, df_civ_2001_metro, df_civ_2001_micro]], ignore_index=True)

mil_long['GeoFips'] = mil_long['GeoFips'].astype(str)
civ_long['GeoFips'] = civ_long['GeoFips'].astype(str)
mil_long = mil_long.dropna(subset=['Year'])  # Remove rows with NaN years
civ_long = civ_long.dropna(subset=['Year'])
# do the same for 'Year'
mil_long['Year'] = mil_long['Year'].astype(int)
civ_long['Year'] = civ_long['Year'].astype(int)


# 2. Merge
merged = pd.merge(civ_long, mil_long, on=['GeoFips', 'GeoName', 'Year'], how='outer')

# 3. Total employment
merged['total_emp'] = merged['civilian_emp'].fillna(0) + merged['military_emp'].fillna(0)

# 4. Remove unmatched
final = pd.merge(
    civ_long, mil_long, on=['GeoFips', 'Year'], how='inner', suffixes=('_civ', '_mil')
)
final['GeoName'] = final['GeoName_civ'] 
final['total_emp'] = final['civilian_emp'] + final['military_emp']
final = final[['GeoFips', 'GeoName', 'Year', 'military_emp', 'civilian_emp', 'total_emp']]
print(final.shape)

final.head()

(80751, 6)


,GeoFips,GeoName,Year,military_emp,civilian_emp,total_emp
0,10180,"Abilene, TX (Metropolitan Statistical Area)",1990,NaN,1534.0,NaN
1,10180,"Abilene, TX (Metropolitan Statistical Area)",1990,80942.0,1534.0,82476.0
2,10180,"Abilene, TX (Metropolitan Statistical Area)",1990,5715.0,1534.0,7249.0
3,10420,"Akron, OH (Metropolitan Statistical Area)",1990,NaN,2399.0,NaN
4,10420,"Akron, OH (Metropolitan Statistical Area)",1990,337124.0,2399.0,339523.0


2. Summaries
- 2.1 Now that your data is loaded and parsed, show the top 5 MSAs by share of military employment,
by share of civilian government employment, and by ratio of military to civilian government em-
ployment. Look at these rankings for the average values between 1990 and 1999, and for 2009
through 2018

In [20]:
# Ensure data is numeric
for col in ['military_emp', 'civilian_emp', 'total_emp']:
    final[col] = pd.to_numeric(final[col], errors='coerce')

def summarize_period(df, years, label):
    period = df[df['Year'].between(years[0], years[1])].copy()
    # Calculate period averages for each MSA
    avg = period.groupby(['GeoFips', 'GeoName']).agg({
        'military_emp': 'mean',
        'civilian_emp': 'mean',
        'total_emp': 'mean'
    }).reset_index()
    avg['mil_share'] = avg['military_emp'] / avg['total_emp']
    avg['civ_share'] = avg['civilian_emp'] / avg['total_emp']
    avg['mil_civ_ratio'] = avg['military_emp'] / avg['civilian_emp']

    print(f"--- {label} ---")
    print("Top 5 by share of military employment:")
    display(avg.sort_values('mil_share', ascending=False).head(5)[['GeoName', 'mil_share']])

    print("\nTop 5 by share of civilian government employment:")
    display(avg.sort_values('civ_share', ascending=False).head(5)[['GeoName', 'civ_share']])

    print("\nTop 5 by ratio of military to civilian employment:")
    display(avg.sort_values('mil_civ_ratio', ascending=False).head(5)[['GeoName', 'mil_civ_ratio']])
    print("\n\n")

# For 1990-1999
summarize_period(final, (1990, 1999), "1990–1999")

# For 2009-2018
summarize_period(final, (2009, 2018), "2009–2018")


--- 1990–1999 ---
Top 5 by share of military employment:


,GeoName,mil_share
255,"Elkhart-Goshen, IN (Metropolitan Statistical A...",0.995242
465,"Laurinburg, NC (Micropolitan Statistical Area)",0.993944
523,"Martinsville, VA (Micropolitan Statistical Are...",0.993916
109,"Breckenridge, CO (Micropolitan Statistical Area)",0.993864
679,"Port Lavaca, TX (Micropolitan Statistical Area)",0.993752



Top 5 by share of civilian government employment:


,GeoName,civ_share
878,"Warner Robins, GA (Metropolitan Statistical Area)",0.289453
110,"Bremerton-Silverdale-Port Orchard, WA (Metropo...",0.218417
884,"Washington-Arlington-Alexandria, DC-VA-MD-WV (...",0.192831
214,"Dayton, TN (Micropolitan Statistical Area)",0.178455
130,"California-Lexington Park, MD (Metropolitan St...",0.175671



Top 5 by ratio of military to civilian employment:


...

,GeoName,mil_share
28,"Andrews, TX (Micropolitan Statistical Area)",0.996761
255,"Elkhart-Goshen, IN (Metropolitan Statistical A...",0.996303
716,"Rockport, TX (Micropolitan Statistical Area)",0.996194
613,"Odessa, TX (Metropolitan Statistical Area)",0.996103
109,"Breckenridge, CO (Micropolitan Statistical Area)",0.995936



Top 5 by share of civilian government employment:


,GeoName,civ_share
878,"Warner Robins, GA (Metropolitan Statistical Area)",0.245029
130,"California-Lexington Park, MD (Metropolitan St...",0.225833
820,"Susanville, CA (Micropolitan Statistical Area)",0.221034
110,"Bremerton-Silverdale-Port Orchard, WA (Metropo...",0.203758
637,"Ozark, AL (Micropolitan Statistical Area)",0.168423



Top 5 by ratio of military to civilian employment:


,GeoName,mil_civ_ratio
28,"Andrews, TX (Micropolitan Statistical Area)",307.750000
255,"Elkhart-Goshen, IN (Metropolitan Statistical A...",269.479911
716,"Rockport, TX (Micropolitan Statistical Area)",261.769231
613,"Odessa, TX (Metropolitan Statistical Area)",255.632181
109,"Breckenridge, CO (Micropolitan Statistical Area)",245.061151


4. Voting Data
- 4.1 We will now look at how military and civilian government share of employment relate to voting
patterns. To begin, we will load in data on the 2018 US House votes by county from the MIT
Election Lab. You can download that file directly, or from Canvas (HOUSE precinct general.csv).
- Explore the data, and look at the code book on the website.
- The only columns we need are the county FIPS codes, the simplified party, and the vote totals.
- What should you do with NaNs? Look up what “OVER VOTES” and “UNDER VOTES”
mean.
    - Ans: 
    - OVER VOTES: More than one candidate selected on a ballot; usually a very small number. Not a party, usually should be dropped.
    - UNDER VOTES: Ballots with no selection for this office. Not a party; also drop.
    - NaNs in 'votes': Should be set to 0 if you pivot, or rows can be dropped before aggregating.
    -> Drop rows where party_simplified is OVER VOTES or UNDER VOTES.
    -> Fill missing votes with 0.

- What uniquely identifies each observation in this data? We want the data to have party
(REPUBLICAN, DEMOCRAT, etc) as the columns with values, while each row is uniquely
identified by county. Use groupby and pivot to do this. You should have 3,150 rows and 6
columns.

    - Ans: Each observation is uniquely identified by the county FIPS code and party.

- What should you do with the NaNs in this reshaped DataFrame?

    - After pivoting, fill all NaNs with 0, since this means that party received zero votes in that county.

In [19]:
votes = pd.read_csv("HOUSE_precinct_general.csv")

votes = votes[['county_fips', 'party_simplified', 'votes']]
votes = votes[~votes['party_simplified'].isin(['OVER VOTES', 'UNDER VOTES'])]
votes = votes.dropna(subset=['county_fips', 'party_simplified'])
votes['votes'] = pd.to_numeric(votes['votes'], errors='coerce').fillna(0)

print(votes.head())
print(votes.columns)

/var/folders/wm/yrd9rf7n58q84c2y8xz4vfjw0000gn/T/ipykernel_6187/2035931307.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv("HOUSE_precinct_general.csv")


   county_fips party_simplified  votes
2         1097         DEMOCRAT    157
3         1097       REPUBLICAN    605
6         1097         DEMOCRAT    148
7         1097       REPUBLICAN    554
11        1097         DEMOCRAT    166
Index(['county_fips', 'party_simplified', 'votes'], dtype='object')


In [23]:
#Group and sum votes
votes_grouped = votes.groupby(['county_fips', 'party_simplified'])['votes'].sum().reset_index()
votes_wide = votes_grouped.pivot(index='county_fips', columns='party_simplified', values='votes').reset_index()
votes_wide.columns.name = None
votes_wide['county_fips'] = votes_wide['county_fips'].astype(str)

# Fill missing values with 0 (this handles counties that don't have votes for certain parties)
votes_wide = votes_wide.fillna(0)

votes_wide = votes_wide.drop_duplicates(subset=['county_fips'])
votes_wide = votes_wide.sort_values('county_fips').reset_index(drop=True)
print(votes_wide.shape)

(3087, 6)
